## Multiclass classification to Binary Classification
### Training
Instead of train a multiclass classifier model which has too many distinct classes, we convert it to a
binary classification problem. That is, for each event, we generated the features associated with the 
place_id(aggregated features), and use {1, 0} labels. To do this we have to solve two problems:
  1. Negative samples. We can use KNN to generate negative samples.
  2. Features. The features should be related to the place_id, such as aggregated features: #checkin aggregated by (place_id, hour range), (place_id, position ramge) and so on.
  
### Basic Statistics
  1. #Training samples: 29,118,021


In [15]:
# Statistics
import pandas as pd

dfTrain = pd.read_csv('data/train.csv')
dfTest = pd.read_csv('data/test.csv')

xMin = dfTrain.x.min()
xMax = dfTrain.x.max()
yMin = dfTrain.y.min()
yMax = dfTrain.y.max()
print "xMin: %f, xMax: %f, yMin: %f, yMax: %f" % (xMin, xMax, yMin, xMax)

xMin: 0.000000, xMax: 10.000000, yMin: 0.000000, yMax: 10.000000


In [16]:
print dfTrain.shape

(29118021, 6)


In [2]:
from sklearn.neighbors import KDTree
dfAll = pd.read_csv('data/train.csv.100000')

def preProcess(df):
    df['x_1000'] = df.apply(lambda x : int(x['x'] * 1000), axis=1) 
    df['x_100'] = df.apply(lambda x : int(x['x'] * 100), axis=1)
    df['y_1000'] = df.apply(lambda x : int(x['y'] * 1000), axis=1) 
    df['y_100'] = df.apply(lambda x : int(x['y'] * 100), axis=1)
    df['timeH'] = df.apply(lambda x: int(x['time'] / 3600) % 24 / 24.000 * 5, axis = 1)
    
def genCandidates(df, topK = 5):
    print 'genCandidates'
    
    tree = KDTree(dfAll[['x', 'y','timeH']])
    _, ind = tree.query(df[['x','y','timeH']], k = topK)
    lstDF = []
    for i in range(topK):
        dfTmp = df.copy()
        ind1 = [x[i] for x in ind]
        #print ind1
        dfTmp['place_id_cand'] = dfAll.iloc[ind1].place_id.values
        lstDF.append(dfTmp)
    df['place_id_cand'] = df['place_id']
    lstDF.append(df)
    df = pd.concat(lstDF, ignore_index = True)
    print len(df.index)
    #df['label'] = df['place_id'] == df['place_id_cand'] ? 1 : 0
    df['label'] = df.apply(lambda x : 1 if x['place_id'] == x['place_id_cand'] else 0, axis=1)  
    #df[['row_id', 'place_id']].to_csv('submission_time.gz', index = False, compression = 'gzip')
    return df

In [3]:
def genFeatures(df, lstTopK = [5]):
    # Depreciated, will not use KNN statis as features since too time consuming
    key = ['x', 'y']
    treeXY = KDTree(dfAll[key])
    for topK in lstTopK:
        print 'top%d_xy' % (topK)
        _, lstTopKIndex = treeXY.query(df[key], k = topK)
        df['top%d_xy' % (topK)] = [[dfAll.iloc[xx].place_id for xx in x].count(df.iloc[index].place_id_cand) for index, x in enumerate(lstTopKIndex)]
    key = ['x', 'y', 'timeH']
    treeXYTimeH = KDTree(dfAll[key])  
    for topK in lstTopK:
        print 'top%d_xyTimeH' % (topK)
        _, lstTopKIndex = treeXYTimeH.query(df[key], k = topK)
        df['top%d_xyTimeH' % (topK)] = [[dfAll.iloc[xx].place_id for xx in x].count(df.iloc[index].place_id_cand) for index, x in enumerate(lstTopKIndex)]
    
    print [dfAll.iloc[xx].place_id for xx in x]
    print df.iloc[index].place_id_cand


dictFeaAggKey = {
    'xy1000_placeId': ['x_1000', 'y_1000', 'place_id_cand'],
    'xy100_placeId': ['x_100', 'y_100', 'place_id_cand'],
    'xy1000_placeId_timeH' : ['x_1000', 'y_1000', 'place_id_cand', 'timeH'],
    'xy100_placeId_timeH' : ['x_100', 'y_100', 'place_id_cand', 'timeH'],
}

dictFeaAggDF = {}

def genFeaAgg(df):
    # The KDTree based feature is too time consuming
    df['place_id_cand'] = df['place_id']
    for fea in dictFeaAggKey.keys():
        keyAgg = dictFeaAggKey[fea]
        dfGroup = df.groupby(keyAgg).count().reset_index()
        dfGroup[fea] = dfGroup['x']
        dfGroup = dfGroup[keyAgg + [fea]]
        dictFeaAggDF[fea] = dfGroup

def MergeAggFea(df):
    dfMerge = df.copy()
    for fea in dictFeaAggDF.keys():
        dfAgg = dictFeaAggDF[fea]
        dfMerge = pd.merge(dfMerge, dfAgg, how = 'left', on = dictFeaAggKey[fea])
    return dfMerge

In [22]:
import random, time
dfAll = pd.read_csv('data/train.csv')
uniqRowID = dfAll.row_id.unique()
print 'preProcess'
timeStart = time.time()
preProcess(dfAll)
timeCurrent = time.time()
trainThres = int(len(uniqRowID) * 0.7)
dfTrain = dfAll.loc[lambda df: df.row_id <= trainThres, :]
dfTest = dfAll.loc[lambda df: df.row_id > trainThres, :]
print 'dfTrainCan'
dfTrainCan = genCandidates(dfTrain)
dfTrainCan = dfTrainCan.drop_duplicates()
print 'dfTestCan'
dfTestCan = genCandidates(dfTest)
dfTestCan = dfTestCan.drop_duplicates()
print 'genFeaAgg'
genFeaAgg(dfTrain)
print 'dfTrainAggFea'
dfTrainAggFea = MergeAggFea(dfTrainCan)
print 'dfTrainAggFea'
dfTestAggFea = MergeAggFea(dfTestCan)
timeCurrent = time.time()
print "%f s has passed" % (timeCurrent - timeStart)

preProcess
dfTrainCan
genCandidates


MemoryError: 

In [6]:
from sklearn.ensemble import RandomForestClassifier
dfTrainAggFea = dfTrainAggFea.fillna(0)
clf = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=0.1)
predictors = ['x', 'y', 'accuracy', 'time', 'xy100_placeId', 'xy1000_placeId', 'xy1000_placeId_timeH', 'xy100_placeId_timeH']
#predictors = ['x', 'y', 'accuracy', 'time', 'xy100_placeId']
clf.fit(dfTrainAggFea[predictors], dfTrainAggFea['label'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.1, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
dfTestAggFea = dfTestAggFea.fillna(0)
lstScore = clf.predict_proba(dfTestAggFea[predictors])
print lstScore[:10]

[[ 0.97904865  0.02095135]
 [ 0.9854385   0.0145615 ]
 [ 0.98960796  0.01039204]
 [ 0.9832609   0.0167391 ]
 [ 0.98122512  0.01877488]
 [ 0.98964386  0.01035614]
 [ 0.98556752  0.01443248]
 [ 0.98765308  0.01234692]
 [ 0.9897      0.0103    ]
 [ 0.99375383  0.00624617]]


In [10]:
def genTopResult(df, lstScore, topK = 3):
    lstRes = []
    idx = -1
    dictRowIdRes = {}
    for _, row in df.iterrows():
        idx += 1
        rowId = row['row_id']
        placIdCan = row['place_id_cand']
        score = lstScore[idx][1]
        dictRowIdRes.setdefault(rowId, [])
        dictRowIdRes[rowId].append([placIdCan, score])
    rowIdTestUnique = sorted(df.row_id.unique())
    for rowId in rowIdTestUnique:
        #print 'dictRowIdRes[rowId]', dictRowIdRes[rowId]
        res = sorted(dictRowIdRes[rowId], key = lambda x: x[1], reverse = True)
        #print 'res', res
        lstRes.append([rowId, [x[0] for x in res[:topK]], res])
    return lstRes

def genLabel(df):
    lstLabel = []
    rowIdTestUnique = sorted(df.row_id.unique())
    for rowId in rowIdTestUnique:
        label = dfTest[dfTest.row_id == rowId].place_id.unique()[0]
        lstLabel.append([label])
    return lstLabel

In [11]:
lstRes0 = genTopResult(dfTestAggFea, lstScore)
lstRes = [map(int, x[1]) for x in lstRes0]
lstLabel = genLabel(dfTestAggFea)

In [12]:
import ml_metrics as metrics
metrics.mapk(lstLabel, lstRes, k = 3)

0.99621654055135167

In [14]:
import numpy as np
importances = clf.feature_importances_
print len(importances)
print len(predictors)


indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(len(predictors)):
    print("%d. %str: %f" % (f + 1, predictors[f], importances[f]))


8
8
Feature ranking:
1. xtr: 0.003726
2. ytr: 0.001115
3. accuracytr: 0.003453
4. timetr: 0.001714
5. xy100_placeIdtr: 0.159998
6. xy1000_placeIdtr: 0.269998
7. xy1000_placeId_timeHtr: 0.289998
8. xy100_placeId_timeHtr: 0.269999
